In [1]:
#
# LSTM모델을 이용해서 주가예측
#
import os
import settings
import pandas as pd        # 라이브러리
import numpy as np         # 라이브러리
from keras.models import Sequential      # 딥러닝을 구동하는 데 필요한 케라스 함수
from keras.layers import Dense           # 딥러닝을 구동하는 데 필요한 케라스 함수
from keras.layers import LSTM            # 딥러닝을 구동하는 데 필요한 케라스 함수
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import math
from sklearn.metrics import mean_squared_error
import pymysql         # 파이썬에서 mysql연동시켜주는 라이브러리


#
# DB테이블 값 조회 (SELECT)
#
connection = pymysql.connect(host='222.122.86.187', port=3306, user='geniuses777', password='stock7840',
                       db='geniuses777', charset='utf8')

# 오차율 가져오기
try:
    with connection.cursor() as cursor:
        sql = "select accuracy from stock_hye WHERE company_name='gold'"
        cursor.execute("set names utf8")
        cursor.execute(sql)
        result_accuracy = cursor.fetchone()
        
        for i in result_accuracy:
            accuracy = i
            
finally:
    connection.close()


# 데이터셋 생성 함수
look_back = 1
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i + look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)
 
#
# 저장되어있는 주식데이터 불러오기
#
sydtpath = os.path.join(settings.BASE_DIR, 'chart_data/%s' % (settings.get_today_str()))
stock_code = "gold"
fullpath = sydtpath + os.path.sep + stock_code + '.csv'
pandf = pd.read_csv(fullpath, index_col="DATE")
pandf = pandf.dropna(how="any")

# 데이터 전처리
nparr = pandf['GOLDAMGBD228NLBM'].values[1:]     # 맨처음 'Close'데이터부터 차례대로 nparr에 저장
print(nparr)
nparr.astype('float32')    # float형으로 변환
print(nparr)
nparr = nparr.reshape(-1,1)
print(nparr)
 
# 정규화 (0~1사이의 값으로 바꿔준다)
scaler = MinMaxScaler(feature_range=(0, 1))
nptf = scaler.fit_transform(nparr)
 
# 학습용, 테스트용 데이터로 나누기 (90%를 학습용 데이터, 10%를 테스트용 데이터)
train_size = int(len(nptf) * 0.9)
test_size = len(nptf) - train_size
train, test = nptf[0:train_size], nptf[train_size:len(nptf)]
print(len(train), len(test))
 
# 학습을 위한 데이터셋 생성하기 (학습용, 테스트용으로 구분)
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
 
# RNN모델은 3차원 데이터
# trainX, testX값을 [samples, time steps, features] 형태로 reshape
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
 
# LSTM모델
model = Sequential()                                                # 딥러닝 구조, 층을 설정
model.add(LSTM(20, input_shape=(1, 1)))     # (timestep, feature)   # 층이 추가됨 (add)   # 입력층, 첫번째 은닉층                   
model.add(Dense(1))                                                 # 출력층 (하나)
# 모델을 컴파일 (컴퓨터가 알아들을 수 있도록)   # 오차함수, 최적화 방법
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])   # metrics : 모델 수행 결과를 나타내게 설정 (과적합 문제 방지)
model.fit(trainX, trainY, epochs=1000, batch_size=400, verbose=2)      # 모델을 실제로 수행    # batch_size : 전체 데이터를 400개씩 사용
                                                                                               # verbose(로깅)  2 : epoch당 나오게

# 예측값 평가하기 (얼마나 정확한지)
testPredict = model.predict(testX)
testPredict = scaler.inverse_transform(testPredict)                 # testPredict : 예측 값
testY = scaler.inverse_transform(testY)                             # testY : 실제 값
testScore = math.sqrt(mean_squared_error(testY, testPredict))       # mean_squared_error : 평균 제곱근 오차
print('Train Score: %.2f RMSE' % testScore)                         # 예측 값과 실제 값 차이 출력
 
# 예측 데이터 출력
lastX = nptf[-1]
lastX = np.reshape(lastX, (1, 1, 1))
lastY = model.predict(lastX)
predict = scaler.inverse_transform(lastY)                    # 정규화 시킨 값을 역변환
print('Predict the Gold value of final day: %d' % predict)  # 데이터 입력 마지막 다음날 종가 예측
    
# 차트출력, 저장
font_path = "C:/Windows/Fonts/a옛날목욕탕L.ttf"
fontprop = fm.FontProperties(fname=font_path, size=15)

plt.plot(testPredict)
plt.plot(testY)

plt.title('[금] 예측 그래프\n예측값 : %d \t\t\t\t\t\t\t\t오차율 : %s' % (predict, accuracy), fontproperties=fontprop)
#plt.savefig("./chart_picture/cj.png",dpi=300)
plt.savefig("C:\source\GOLD\chart_picture\gold.png", dpi=300)
#plt.show()

# 사진 데이터 binary형식으로 바꿔주는 함수
def convertToBinaryData(filename):
    #Convert digital data to binary format
    with open(filename, 'rb') as file:
        binaryData = file.read()
    return binaryData


# 
# DB테이블 값 삽입 (INSERT)
#
# MySQL Connection 연결
connection = pymysql.connect(host='222.122.86.187', port=3306, user='geniuses777', password='stock7840',
                       db='geniuses777', charset='utf8')
try:
    with connection.cursor() as cursor:
        sql = 'INSERT INTO stock_hye (company_name, stock_price, image) VALUES (%s, %s, %s) ON DUPLICATE KEY UPDATE stock_price = VALUES(stock_price), image = VALUES(image)'
        image = convertToBinaryData("C:\source\SPF\chart_picture\cj.png")
        cursor.execute(sql, ('gold', int(predict), image))          # 넣으려는 값
    connection.commit()
    
finally:
    connection.close()

Using TensorFlow backend.


[ 282.1   280.35  282.   ... 1484.45 1490.85 1487.45]
[ 282.1   280.35  282.   ... 1484.45 1490.85 1487.45]
[[ 282.1 ]
 [ 280.35]
 [ 282.  ]
 ...
 [1484.45]
 [1490.85]
 [1487.45]]
4502 501
Epoch 1/1000
 - 1s - loss: 0.2152 - acc: 2.2222e-04
Epoch 2/1000
 - 0s - loss: 0.1827 - acc: 2.2222e-04
Epoch 3/1000
 - 0s - loss: 0.1542 - acc: 2.2222e-04
Epoch 4/1000
 - 0s - loss: 0.1293 - acc: 2.2222e-04
Epoch 5/1000
 - 0s - loss: 0.1081 - acc: 2.2222e-04
Epoch 6/1000
 - 0s - loss: 0.0901 - acc: 2.2222e-04
Epoch 7/1000
 - 0s - loss: 0.0753 - acc: 2.2222e-04
Epoch 8/1000
 - 0s - loss: 0.0633 - acc: 2.2222e-04
Epoch 9/1000
 - 0s - loss: 0.0541 - acc: 2.2222e-04
Epoch 10/1000
 - 0s - loss: 0.0475 - acc: 2.2222e-04
Epoch 11/1000
 - 0s - loss: 0.0426 - acc: 4.4444e-04
Epoch 12/1000
 - 0s - loss: 0.0391 - acc: 4.4444e-04
Epoch 13/1000
 - 0s - loss: 0.0366 - acc: 4.4444e-04
Epoch 14/1000
 - 0s - loss: 0.0344 - acc: 4.4444e-04
Epoch 15/1000
 - 0s - loss: 0.0325 - acc: 4.4444e-04
Epoch 16/1000
 - 0s - los

Epoch 144/1000
 - 0s - loss: 8.8607e-05 - acc: 4.4444e-04
Epoch 145/1000
 - 0s - loss: 8.8133e-05 - acc: 4.4444e-04
Epoch 146/1000
 - 0s - loss: 8.7585e-05 - acc: 4.4444e-04
Epoch 147/1000
 - 0s - loss: 8.7153e-05 - acc: 4.4444e-04
Epoch 148/1000
 - 0s - loss: 8.6639e-05 - acc: 4.4444e-04
Epoch 149/1000
 - 0s - loss: 8.6208e-05 - acc: 4.4444e-04
Epoch 150/1000
 - 0s - loss: 8.5694e-05 - acc: 4.4444e-04
Epoch 151/1000
 - 0s - loss: 8.5355e-05 - acc: 4.4444e-04
Epoch 152/1000
 - 0s - loss: 8.4862e-05 - acc: 4.4444e-04
Epoch 153/1000
 - 0s - loss: 8.4238e-05 - acc: 4.4444e-04
Epoch 154/1000
 - 0s - loss: 8.3824e-05 - acc: 4.4444e-04
Epoch 155/1000
 - 0s - loss: 8.3299e-05 - acc: 4.4444e-04
Epoch 156/1000
 - 0s - loss: 8.2819e-05 - acc: 4.4444e-04
Epoch 157/1000
 - 0s - loss: 8.2369e-05 - acc: 4.4444e-04
Epoch 158/1000
 - 0s - loss: 8.1899e-05 - acc: 4.4444e-04
Epoch 159/1000
 - 0s - loss: 8.1516e-05 - acc: 4.4444e-04
Epoch 160/1000
 - 0s - loss: 8.0982e-05 - acc: 4.4444e-04
Epoch 161/1000

 - 0s - loss: 5.2667e-05 - acc: 4.4444e-04
Epoch 286/1000
 - 0s - loss: 5.2568e-05 - acc: 4.4444e-04
Epoch 287/1000
 - 0s - loss: 5.2556e-05 - acc: 4.4444e-04
Epoch 288/1000
 - 0s - loss: 5.2516e-05 - acc: 4.4444e-04
Epoch 289/1000
 - 0s - loss: 5.2455e-05 - acc: 4.4444e-04
Epoch 290/1000
 - 0s - loss: 5.2530e-05 - acc: 4.4444e-04
Epoch 291/1000
 - 0s - loss: 5.2372e-05 - acc: 4.4444e-04
Epoch 292/1000
 - 0s - loss: 5.2369e-05 - acc: 4.4444e-04
Epoch 293/1000
 - 0s - loss: 5.2355e-05 - acc: 4.4444e-04
Epoch 294/1000
 - 0s - loss: 5.2313e-05 - acc: 4.4444e-04
Epoch 295/1000
 - 0s - loss: 5.2367e-05 - acc: 4.4444e-04
Epoch 296/1000
 - 0s - loss: 5.2232e-05 - acc: 4.4444e-04
Epoch 297/1000
 - 0s - loss: 5.2243e-05 - acc: 4.4444e-04
Epoch 298/1000
 - 0s - loss: 5.2228e-05 - acc: 4.4444e-04
Epoch 299/1000
 - 0s - loss: 5.2151e-05 - acc: 4.4444e-04
Epoch 300/1000
 - 0s - loss: 5.2170e-05 - acc: 4.4444e-04
Epoch 301/1000
 - 0s - loss: 5.2183e-05 - acc: 4.4444e-04
Epoch 302/1000
 - 0s - loss: 

Epoch 427/1000
 - 0s - loss: 5.1551e-05 - acc: 4.4444e-04
Epoch 428/1000
 - 0s - loss: 5.1580e-05 - acc: 4.4444e-04
Epoch 429/1000
 - 0s - loss: 5.1531e-05 - acc: 4.4444e-04
Epoch 430/1000
 - 0s - loss: 5.1661e-05 - acc: 4.4444e-04
Epoch 431/1000
 - 0s - loss: 5.1522e-05 - acc: 4.4444e-04
Epoch 432/1000
 - 0s - loss: 5.1495e-05 - acc: 4.4444e-04
Epoch 433/1000
 - 0s - loss: 5.1529e-05 - acc: 4.4444e-04
Epoch 434/1000
 - 0s - loss: 5.1514e-05 - acc: 4.4444e-04
Epoch 435/1000
 - 0s - loss: 5.1599e-05 - acc: 4.4444e-04
Epoch 436/1000
 - 0s - loss: 5.2108e-05 - acc: 4.4444e-04
Epoch 437/1000
 - 0s - loss: 5.2069e-05 - acc: 4.4444e-04
Epoch 438/1000
 - 0s - loss: 5.1636e-05 - acc: 4.4444e-04
Epoch 439/1000
 - 0s - loss: 5.1553e-05 - acc: 4.4444e-04
Epoch 440/1000
 - 0s - loss: 5.1561e-05 - acc: 4.4444e-04
Epoch 441/1000
 - 0s - loss: 5.1638e-05 - acc: 4.4444e-04
Epoch 442/1000
 - 0s - loss: 5.1651e-05 - acc: 4.4444e-04
Epoch 443/1000
 - 0s - loss: 5.1609e-05 - acc: 4.4444e-04
Epoch 444/1000

 - 0s - loss: 5.1892e-05 - acc: 4.4444e-04
Epoch 569/1000
 - 0s - loss: 5.2163e-05 - acc: 4.4444e-04
Epoch 570/1000
 - 0s - loss: 5.1821e-05 - acc: 4.4444e-04
Epoch 571/1000
 - 0s - loss: 5.1714e-05 - acc: 4.4444e-04
Epoch 572/1000
 - 0s - loss: 5.1329e-05 - acc: 4.4444e-04
Epoch 573/1000
 - 0s - loss: 5.2020e-05 - acc: 4.4444e-04
Epoch 574/1000
 - 0s - loss: 5.2010e-05 - acc: 4.4444e-04
Epoch 575/1000
 - 0s - loss: 5.1647e-05 - acc: 4.4444e-04
Epoch 576/1000
 - 0s - loss: 5.1681e-05 - acc: 4.4444e-04
Epoch 577/1000
 - 0s - loss: 5.1815e-05 - acc: 4.4444e-04
Epoch 578/1000
 - 0s - loss: 5.1622e-05 - acc: 4.4444e-04
Epoch 579/1000
 - 0s - loss: 5.1479e-05 - acc: 4.4444e-04
Epoch 580/1000
 - 0s - loss: 5.1561e-05 - acc: 4.4444e-04
Epoch 581/1000
 - 0s - loss: 5.1771e-05 - acc: 4.4444e-04
Epoch 582/1000
 - 0s - loss: 5.2059e-05 - acc: 4.4444e-04
Epoch 583/1000
 - 0s - loss: 5.1784e-05 - acc: 4.4444e-04
Epoch 584/1000
 - 0s - loss: 5.1470e-05 - acc: 4.4444e-04
Epoch 585/1000
 - 0s - loss: 

Epoch 710/1000
 - 0s - loss: 5.2464e-05 - acc: 4.4444e-04
Epoch 711/1000
 - 0s - loss: 5.3528e-05 - acc: 4.4444e-04
Epoch 712/1000
 - 0s - loss: 5.3161e-05 - acc: 4.4444e-04
Epoch 713/1000
 - 0s - loss: 5.2035e-05 - acc: 4.4444e-04
Epoch 714/1000
 - 0s - loss: 5.1410e-05 - acc: 4.4444e-04
Epoch 715/1000
 - 0s - loss: 5.1928e-05 - acc: 4.4444e-04
Epoch 716/1000
 - 0s - loss: 5.2867e-05 - acc: 4.4444e-04
Epoch 717/1000
 - 0s - loss: 5.3728e-05 - acc: 4.4444e-04
Epoch 718/1000
 - 0s - loss: 5.3058e-05 - acc: 4.4444e-04
Epoch 719/1000
 - 0s - loss: 5.1697e-05 - acc: 4.4444e-04
Epoch 720/1000
 - 0s - loss: 5.3066e-05 - acc: 4.4444e-04
Epoch 721/1000
 - 0s - loss: 5.1532e-05 - acc: 4.4444e-04
Epoch 722/1000
 - 0s - loss: 5.1419e-05 - acc: 4.4444e-04
Epoch 723/1000
 - 0s - loss: 5.1254e-05 - acc: 4.4444e-04
Epoch 724/1000
 - 0s - loss: 5.1314e-05 - acc: 4.4444e-04
Epoch 725/1000
 - 0s - loss: 5.1299e-05 - acc: 4.4444e-04
Epoch 726/1000
 - 0s - loss: 5.1657e-05 - acc: 4.4444e-04
Epoch 727/1000

Epoch 852/1000
 - 0s - loss: 5.1725e-05 - acc: 4.4444e-04
Epoch 853/1000
 - 0s - loss: 5.2142e-05 - acc: 4.4444e-04
Epoch 854/1000
 - 0s - loss: 5.1896e-05 - acc: 4.4444e-04
Epoch 855/1000
 - 0s - loss: 5.3812e-05 - acc: 4.4444e-04
Epoch 856/1000
 - 0s - loss: 5.2449e-05 - acc: 4.4444e-04
Epoch 857/1000
 - 0s - loss: 5.1569e-05 - acc: 4.4444e-04
Epoch 858/1000
 - 0s - loss: 5.1881e-05 - acc: 4.4444e-04
Epoch 859/1000
 - 0s - loss: 5.1201e-05 - acc: 4.4444e-04
Epoch 860/1000
 - 0s - loss: 5.2019e-05 - acc: 4.4444e-04
Epoch 861/1000
 - 0s - loss: 5.1453e-05 - acc: 4.4444e-04
Epoch 862/1000
 - 0s - loss: 5.1565e-05 - acc: 4.4444e-04
Epoch 863/1000
 - 0s - loss: 5.2116e-05 - acc: 4.4444e-04
Epoch 864/1000
 - 0s - loss: 5.2131e-05 - acc: 4.4444e-04
Epoch 865/1000
 - 0s - loss: 5.2974e-05 - acc: 4.4444e-04
Epoch 866/1000
 - 0s - loss: 5.2627e-05 - acc: 4.4444e-04
Epoch 867/1000
 - 0s - loss: 5.5025e-05 - acc: 4.4444e-04
Epoch 868/1000
 - 0s - loss: 5.2611e-05 - acc: 4.4444e-04
Epoch 869/1000

 - 0s - loss: 5.1233e-05 - acc: 4.4444e-04
Epoch 994/1000
 - 0s - loss: 5.1173e-05 - acc: 4.4444e-04
Epoch 995/1000
 - 0s - loss: 5.1408e-05 - acc: 4.4444e-04
Epoch 996/1000
 - 0s - loss: 5.1141e-05 - acc: 4.4444e-04
Epoch 997/1000
 - 0s - loss: 5.5453e-05 - acc: 4.4444e-04
Epoch 998/1000
 - 0s - loss: 5.2508e-05 - acc: 4.4444e-04
Epoch 999/1000
 - 0s - loss: 5.1265e-05 - acc: 4.4444e-04
Epoch 1000/1000
 - 0s - loss: 5.2545e-05 - acc: 4.4444e-04
Train Score: 9.08 RMSE
Predict the Gold value of final day: 1486
